In [1]:
ApiKey = ""
ApiSecret = ""

In [2]:
import boto3

session = boto3.Session(
    aws_access_key_id="",
    aws_secret_access_key="/s"
)

In [ ]:
s3_client = session.client('s3', region_name='us-east-1')

# Specify the S3 bucket and file name for temporary storage
bucket_name = 'vonage-hackathon'
document_name = 'Docs/images (2).jpeg'

# Upload the local PDF file to S3
s3_client.upload_file(document_name, bucket_name, document_name)

In [8]:
import requests
import base64

def download_recording(api_key, api_secret, recording_url, destination_path):
    """
    Download a Vonage Video API recording.

    :param api_key: Your Vonage Video API key.
    :param api_secret: Your Vonage Video API secret.
    :param recording_url: URL of the recording to download.
    :param destination_path: Path where the recording will be saved.
    """
    # Construct the authentication header
    auth_header = {'Authorization': 'Basic ' + base64.b64encode((api_key + ':' + api_secret).encode()).decode()}

    # Make a request to download the recording
    response = requests.get(recording_url, headers=auth_header, stream=True)

    # Check if the request was successful (HTTP status code 200)
    if response.status_code == 200:
        # Save the recording to the specified destination
        with open(destination_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
        print(f"Recording downloaded successfully to {destination_path}")
    else:
        print(f"Failed to download recording. HTTP status code: {response.status_code}")

# Example usage
api_key = 'YOUR_API_KEY'
api_secret = 'YOUR_API_SECRET'
recording_url = 'URL_OF_YOUR_RECORDING'
destination_path = 'path/to/save/recording.mp4'

download_recording(api_key, api_secret, recording_url, destination_path)

NameError: name 'base64' is not defined

In [16]:
import time
import requests

def transcribe_audio(video_file_path):
    aws_region = 'us-east-1'

    # Set up the Amazon Transcribe client
    transcribe_client = session.client('transcribe', region_name=aws_region)

    # Start a transcription job
    job_name = f'transcription-job-{int(time.time())}'
    job_uri = f's3://{bucket_name}/{video_file_path}'  # Update with your S3 bucket and video file path
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': job_uri},
        MediaFormat='mp4',
        LanguageCode='en-US'
    )

    # Wait for the transcription job to complete
    while True:
        response = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        if response['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        time.sleep(5)

    # Get the transcription results
    if response['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
        transcript_uri = response['TranscriptionJob']['Transcript']['TranscriptFileUri']
        transcript_response = requests.get(transcript_uri)
        transcript_json = transcript_response.json()
        transcription_text = transcript_json['results']['transcripts'][0]['transcript']
        return transcription_text
    else:
        raise Exception(f"Transcription job failed: {response['TranscriptionJob']['FailureReason']}")

# Example usage
video_file_path = 'Vids/Stomach Pain __ Conversation between Doctor and Patient __ Easy English Conversation(720P_HD).mp4'
transcription_result = transcribe_audio(video_file_path)
print("Transcription Result:")
print(transcription_result)

Transcription Result:
Good evening doctor. Good evening. Please sit down. I've got a stomach ache doctor. I see. What do you feel? I have got sharp pain and heartburn. How long have you had this pain in vicinity for two days? Now? It does not go away. I eat and after every meal, my stomach hurts at night. The pain seems to double up. Is the pain continuous or does it come and go? Occasionally it comes and goes, it hurts me the most after every meal. Other times, the pain isn't that severe. Is there any kind of food that affects you greasy ones and food which has got coconut milk in it? Ok. And we're in the abdomen. Does it hurt? Does the pain travel to your chest or shoulder? It hurts in the middle? Sometimes the pain travels across my abdomen. You said you have heartburn besides the pain too? Right?


In [18]:
import openai

# Set your OpenAI API key
openai.api_key = ''

def extract_information_from_text(prompt):
    # Use the GPT-3 completions API to generate responses based on the prompt
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0,
        max_tokens=150
    )

    return response.choices[0].text.strip()


prompt = f"You are given a conversation between a doctor and a patient, go through it and extract important infromation such asas name, age, gender, diseas, any allergies to a certain medicine, any good to have tests, etc. from the following text:\n{transcription_result}"

extracted_information = extract_information_from_text(prompt)
print(extracted_information)

Yes, I have heartburn too.

Name: Patient
Age: Unknown
Gender: Unknown
Disease: Stomach ache
Allergies: Unknown
Tests: Unknown
